In [1]:
import torch
print(torch.cuda.is_available())  # Use GPU if available

True


In [ ]:
import os
import yaml
import torch
import glob
from ultralytics import YOLO
from IPython.display import display, Image
import matplotlib.pyplot as plt

# Load dataset configuration
with open('Final_Data_After_cleaning/data.yaml', 'r') as file:
    data_config = yaml.safe_load(file)
    
print("Dataset information:")
print(f"Classes: {data_config['names']}")
print(f"Number of classes: {data_config['nc']}")
print(f"Training images path: {data_config['train']}")
print(f"Validation images path: {data_config['val']}")
print(f"Testing images path: {data_config['test']}")

# Create models directory if it doesn't exist
os.makedirs('models', exist_ok=True)

# Check for existing checkpoint to resume training
checkpoint_dir = 'models/fire_smoke_detector'
last_checkpoint = None

if os.path.exists(checkpoint_dir):
    checkpoint_files = glob.glob(os.path.join(checkpoint_dir, 'weights/last*.pt'))
    if checkpoint_files:
        last_checkpoint = max(checkpoint_files, key=os.path.getmtime)
        print(f"Found checkpoint: {last_checkpoint}")

# Initialize YOLO model
try:
    if last_checkpoint and os.path.exists(last_checkpoint):
        print(f"Resuming training from checkpoint: {last_checkpoint}")
        model = YOLO(last_checkpoint)
    else:
        print("Starting new training with YOLOv8n base model")
        model = YOLO('yolo11n.pt')
except FileNotFoundError:
    print("Model file not found. Using YOLOv8n which is available in Ultralytics.")
    model = YOLO('yolo11n.pt')

# Training configuration with reduced workers to avoid DataLoader issues
# training_results = model.train(
#     data='Final_Data_After_cleaning/data.yaml',
#     epochs=5,  # Adjust based on your needs
#     imgsz=640,   # Image size
#     batch=8,     # Reduced batch size for more stability
#     device='cpu',  # Use CPU
#     project='models',
#     name='fire_smoke_detector',
#     patience=20,  # Early stopping patience
#     save=True,   # Save the model after training
#     exist_ok=True,
#     resume=True if last_checkpoint else False,  # Resume training if checkpoint exists
#     workers=0    # Using single process data loading to avoid worker errors
# )

# Validate the model with single process data loading
val_results = model.val(workers=0)
print("Validation Results:", val_results)

# Plot training metrics
try:
    training_results.plot()
    plt.savefig('models/training_metrics.png')
    plt.show()
except Exception as e:
    print(f"Could not plot training metrics: {e}")

# Save the model in different formats for deployment
trained_model = YOLO('models/fire_smoke_detector/weights/best.pt')

# Export to ONNX format for potential deployment optimization
try:
    trained_model.export(format='onnx')
    print("Model exported to ONNX format at: models/fire_smoke_detector/weights/best.onnx")
except Exception as e:
    print(f"ONNX export failed: {e}")

# Test the model on sample images with better error handling
test_images_dir = 'Final_Data_After_cleaning/test/images'
try:
    sample_images = os.listdir(test_images_dir)[:5]  # Testing with 5 sample images

    plt.figure(figsize=(15, 12))
    for i, img in enumerate(sample_images):
        img_path = os.path.join(test_images_dir, img)
        results = trained_model(img_path, workers=0)  # Use single process for prediction
        
        # Plot results
        plt.subplot(3, 2, i+1)
        plt.imshow(results[0].plot())
        plt.title(f"Sample {i+1}: {img}")
        plt.axis('off')
        
    plt.tight_layout()
    plt.savefig('models/sample_predictions.png')
    plt.show()
except Exception as e:
    print(f"Error testing on sample images: {e}")

print("Training complete! Best model saved at: models/fire_smoke_detector/weights/best.pt")

Dataset information:
Classes: ['Fire', 'Smoke']
Number of classes: 2
Training images path: ../train/images
Validation images path: ../valid/images
Testing images path: ../test/images
Found checkpoint: models/fire_smoke_detector\weights\last.pt
Resuming training from checkpoint: models/fire_smoke_detector\weights\last.pt
Ultralytics 8.3.155  Python-3.11.0 torch-2.7.1+cu128 CPU (13th Gen Intel Core(TM) i5-13420H)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=Final_Data_After_cleaning/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=5, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7

train: Scanning D:\AIPPROJECT03\AIPP Zone\Python\Fire Detector YOLOv11\Final_Data_After_cleaning\train\labels.cache... 100000 images, 249 backgrounds, 0 corrupt: 100%|██████████| 100000/100000 [00:00<?, ?it/s]


WARNING Box and segment counts should be equal, but got len(segments) = 837, len(boxes) = 237443. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.
val: Fast image access  (ping: 0.00.0 ms, read: 581.9309.5 MB/s, size: 37.4 KB)


val: Scanning D:\AIPPROJECT03\AIPP Zone\Python\Fire Detector YOLOv11\Final_Data_After_cleaning\valid\labels.cache... 20000 images, 58 backgrounds, 0 corrupt: 100%|██████████| 20000/20000 [00:00<?, ?it/s]

WARNING Box and segment counts should be equal, but got len(segments) = 163, len(boxes) = 47649. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.


Plotting labels to models\fire_smoke_detector\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)


AssertionError: models\fire_smoke_detector\weights\last.pt training to 5 epochs is finished, nothing to resume.
Start a new training without resuming, i.e. 'yolo train model=models\fire_smoke_detector\weights\last.pt'